# Kata for Makemore pt. 3

## Importing packages

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

## Importing data

In [3]:
words = open('../data/names.txt').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

## Build vocabulary and map to/from integers

In [12]:
chars = sorted(list(set(''.join(words))))
itos = {i+1:s for i, s in enumerate(chars)}
itos[0] = '.'
stoi = {s:i for i, s in itos.items()}
vocab_size = len(itos)

print(vocab_size)
print(itos)

27
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


## Build the dataset splits

In [13]:
block_size = 3

def build_dataset(words):
    X, Y = [], []
    for name in words:
        context = [0] * block_size
        for char in name + '.':
            ix = stoi[char]
            X.append(context)
            Y.append(context)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)

    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


torch.Size([182625, 3]) torch.Size([182625, 3])
torch.Size([22655, 3]) torch.Size([22655, 3])
torch.Size([22866, 3]) torch.Size([22866, 3])


## Modules

In [39]:
# Linear layer
class Linear:

    def __init__(self, fan_in, fan_out, bias=True):
        self.weights = torch.randn((fan_in, fan_out)) * 0.5
        self.bias = torch.zeros(fan_out) if bias else None
        self.use_bias = bias

    def __call__(self, x):
        out = x @ self.weights
        if self.use_bias:
            out += self.bias
        return out

    def parameters(self):
        return [self.weights, self.bias]

# Batch normalisation layer
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.gain = torch.ones(dim)
        self.bias = torch.zeros(dim)
        self.mean_running = torch.zeros(dim)
        self.std_running = torch.ones(dim)
        self.training = True

    def __call__(self, x):
        if self.training:
            self.mean_running = (1 - momentum) * self.mean_running + momentum * xmean
            self.std_running = (1 - momentum) * self.std_running + momentum * xstd
        xmean = x.mean(1, keepdim=True)
        xstd = x.std(1, keepdim=True)
        out = self.gain * ((x - xmean) / xstd) + self.bias
        return out

    def parameters(self):
        return [self.gain, self.bias]

# Tanh
class Tanh:

    def __call__(self, x):
        out = torch.tanh(x)
        return out

    def parameters(self):
        return []

## Instantiate the Neural Network

In [40]:
n_emb = 10
n_hidden = 100

C = torch.randn((vocab_size, n_emb))
layers = [
    Linear((block_size * n_emb), n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden), Tanh(),
    Linear(            n_hidden, n_hidden), Tanh(),
    Linear(            n_hidden, vocab_size)
]

## Training loop

In [42]:
max_steps = 2000
batch_size = 32
lossi = []

for i in range(max_steps):

    # Build mini batch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix]
    print(Yb.shape)

    # Forward pass
    out = C[Xb]
    out = out.view(out.shape[0], -1)
    for layer in layers:
         out = layer(out)
    loss = F.cross_entropy(out, Yb)

    # Backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # Update
    lr = 0.1 if max_steps < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # Track stats
    if i % 10000 == 0:
        print(f'{i:7d} / {max_steps}: {loss}')
    lossi.append(loss.log10().item())

torch.Size([32, 3])


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

## Diagnostic tools

### Activation distribution

### Gradient distribution

### Weights gradient activation

### Weights update ratio